In the program, we have the last digit of sid `7`, so the order no. is `7`,$S1$ is 688 *China Overseas*, initial stock price `12.18`, volatility `43.6%`; $S2$ is 857 *Petrochina*, initial stock price `6.03`, volatility `30.0%`; $S1/S2$ correlation coefficient `0.304`, Group 3.

For Group 3, the $F\%=102.6\%, UB\%=130.0\%, A\%=122.0\%$

All codes below were executed on an Intel Core i5-12500H chip Windows platform with 40G of RAM. For the programs below, about 1.5G available RAM is required.

Programmes are executed on Python 3.10, packages of `math`, `scipy`, `numpy` and `tqdm` required.

### Q1(i)
Use the Black-Scholes model to price the following option for stock $S1$:

European put option, at-the-money (*S=K*=price of the stock as of start date), 
continuously compounded interest rate *r = 4.17%p.a*., maturity *T = 0.75* year. 

[Black-Scholes formula (assume dividend yield = 0):

$P=K e^{-rT}N(-d_2)-SN(-d_1)$

where $d_1=\dfrac{\ln{(S/K)}+(r+\sigma^2/2)T}{\sigma\sqrt{T}}, d_2=d_1-\sigma\sqrt{T}$]

In [1]:
import math
from scipy.stats import norm

S = 12.18 # initial stock price
K = 12.18 # given in the qustion, S = K = init price
r = 4.17 / 100 # annualy interest rate (percentage)
sigma = 43.6 / 100 # volatility (percentage)
T = 0.75 # time of maturity in year

d1 = (math.log(S / K) + (r + (sigma ** 2) / 2) * T) / (sigma * math.sqrt(T))
d2 = d1 - sigma * math.sqrt(T)

P = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
print(f"Price of stock S1 is ${P:.3f}.")

Price of stock S1 is $1.614.


### Q1(ii)

Use a Monte Carlo scheme with time steps *N = 150*, i.e. $\Delta t=T/N=1/200$ (refer to the discretization scheme in Topic 1-2, slides 37 and 38; don’t use the exact formula).  Give the answers with: (a) 10000 paths; (b) 300000 paths. 

[Hint: the answers for (ii)(a) and (ii)(b) should be close to the answer in (i) above]

The following is a slow version following the slide 37 and 38 in Topic 1-2, step by step following a large table.

In [2]:
import numpy as np
from tqdm import tqdm

# Given parameters
S0 = 12.18  # initial stock price
K = 12.18   # strike price (at-the-money)
r = 4.17 / 100  # continuously compounded interest rate
sigma = 43.6 / 100  # volatility
N = 150     # number of time steps
dt = 1 / 200 # time step size
T = 0.75 # total time calculated

# Number of paths for the simulation
num_paths_1 = 10_000
num_paths_2 = 300_000

def monte_carlo_option_price(num_paths: int) -> float:
    """
        Calculate the Monte Carlo option price.

        Parameters:
            - num_paths (int): The number of simulation paths to use.

        Returns:
            - float: The estimated price of the option using Monte Carlo simulation.
    """
    S = np.zeros((num_paths, N, 3))
    S[:, 0, 0] = S0 # initial price

    for i in tqdm(range(num_paths)):
        z = np.random.normal(0, 1, N) # generate random sample of epsilon
        S[i, :, 1] = z
        for j in range(N):
            S[i, j, 2] = dt * r * S[i, j, 0] +\
                  sigma * S[i, j, 0] * S[i, j, 1] * np.sqrt(dt) #delta S
            if j != (N - 1): 
                S[i, j + 1, 0] = S[i, j, 0] + S[i, j, 2]

    # Calculate the payoff at maturity
    payoff = np.maximum(K - S[:, N - 1, 0], 0)

    # Discount the average payoff to the present value
    option_price = np.exp(-r * T) * np.mean(payoff)
    return option_price

In [3]:
# Calculate option price for 10,000 and 300,000 paths
option_price_1 = monte_carlo_option_price(num_paths_1)
option_price_2 = monte_carlo_option_price(num_paths_2)

print(f"Answers:\n(a) 10000 paths: ${option_price_1:.3f};\
      \n(b) 300000 paths: ${option_price_2:.3f}")

100%|██████████| 300000/300000 [01:00<00:00, 4962.74it/s]

Answers:
(a) 10000 paths: $1.568;      
(b) 300000 paths: $1.609


The function below executes the same as above but have a faster calculation.

In [4]:
def monte_carlo_option_price_faster(num_paths: int) -> float:
    """
        Calculate the Monte Carlo option price.

        Parameters:
            - num_paths (int): The number of simulation paths to use.

        Returns:
            - float: The estimated price of the option using Monte Carlo simulation.
    """
    S = np.zeros((num_paths))
    S[:] = S0  # initial price

    for _ in tqdm(range(N)):
        z = np.random.normal(0, 1, num_paths) 
        dS = r * S * dt + sigma * S * z * np.sqrt(dt)
        S += dS

    # Calculate the payoff at maturity and discount to present value
    payoff = np.maximum(S0 - S, 0)
    option_price = np.exp(-r * T) * np.mean(payoff)

    return option_price

In [5]:
# Calculate option price for 10,000 and 300,000 paths
option_price_1 = monte_carlo_option_price_faster(num_paths_1)
option_price_2 = monte_carlo_option_price_faster(num_paths_2)

print(f"Answers:\n(a) 10000 paths: ${option_price_1:.3f};\
      \n(b) 300000 paths: ${option_price_2:.3f}")

100%|██████████| 150/150 [00:01<00:00, 118.66it/s]

Answers:
(a) 10000 paths: $1.606;      
(b) 300000 paths: $1.613
